In [7]:
from Reversi import *
import time
import multiprocessing
import numpy as np

In [8]:
from random_player import player as random_AI
from one_layer_1 import player as one_layer_1_AI
from one_layer_2 import player as one_layer_2_AI
from two_layer_1 import player as two_layer_1_AI
from two_layer_2 import player as two_layer_2_AI
from two_layer_3 import player as two_layer_3_AI
from two_layer_4 import player as two_layer_4_AI
from professor import player as professor_AI
from my_code import player as my_AI
from three_layer_1 import player as three_layer_1_AI
from three_layer_2 import player as three_layer_2_AI

In [25]:
from Reversi import *
import multiprocessing
import numpy as np

def Game(player1, player2, first_player):
    board = BoardInit()
    current_player = first_player  # Determine first player

    while True:
        if current_player == Black:
            if random.randint(1, 10) == 20:
                random_AI(current_player, board)
            else:
                x, y = player1(current_player, board)
        else:
            if random.randint(1, 10) == 20:
                random_AI(current_player, board)
            else:
                x, y = player2(current_player, board)

        if (x, y) == (0, 0):  # No valid move
            if all(PossibleMove((-1) * current_player, board) == []):
                break  # No valid moves for either player, game ends
            current_player *= -1  # Switch player
            continue

        # Place the move on the board
        board = PlaceMove(current_player, board, x, y)

        # Check if there are valid moves for the opponent; if none, game over
        if PossibleMove((-1) * current_player, board) == []:
            break

        # Switch player
        current_player *= -1

    # Calculate the final result
    result = 0
    for row in board:
        for cell in row:
            if cell == Black:
                result += 1
            elif cell == White:
                result -= 1

    return result if first_player == Black else -result


def simulate_game(args):
    player1, player2 = args
    # Simulate two games, one where each player starts
    result1 = Game(player1, player2, Black)  # Player1 starts as Black
    result2 = Game(player2, player1, Black)  # Player2 starts as Black
    return (result1, -result2)


def simulate_multiple_games(player1, player2, num_games):
    scores = []

    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        game_results = pool.map(simulate_game, [(player1, player2)] * num_games)

    # Flatten the results since each simulate_game returns two scores
    for score_pair in game_results:
        scores.extend(score_pair)

    return scores


def compare_all_ais(playerA, player_dict, num_games):
    keys = list(player_dict.keys())
    results = {}
    i = 1
    for playerB in keys:
        playerA_ai = player_dict[playerA]
        playerB_ai = player_dict[playerB]
        scores = simulate_multiple_games(playerA_ai, playerB_ai, num_games)
        results[f"{playerA} vs {playerB}"] = scores
        print(f"{i}/{len(keys)} matchups simulated")
        i += 1
    return results

In [26]:
dic = {
"random_AI": random_AI,
"one_layer_1_AI": one_layer_1_AI,
"one_layer_2_AI": one_layer_2_AI,
"two_layer_1_AI": two_layer_1_AI,
"two_layer_2_AI": two_layer_2_AI,
"two_layer_3_AI": two_layer_3_AI,
"two_layer_4_AI": two_layer_4_AI,
"professor_AI": professor_AI,
"my_AI": my_AI,
"three_layer_1_AI": three_layer_1_AI,
# "three_layer_2_AI": three_layer_2_AI
}

In [35]:
playerA = "my_AI"
num_games = 1000  # please use iterations times that is multiples of 12
results = compare_all_ais(playerA, dic, num_games)
# Sorting the match results by average score
sorted_results = sorted(results.items(), key=lambda item: np.mean(item[1]))
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}          [data:{score}]")

1/10 matchups simulated
2/10 matchups simulated
3/10 matchups simulated
4/10 matchups simulated
5/10 matchups simulated
6/10 matchups simulated
7/10 matchups simulated
8/10 matchups simulated
9/10 matchups simulated
10/10 matchups simulated
my_AI vs two_layer_2_AI : -0.63          [data:[17, 22, -28, 4, 22, -7, -22, -24, -16, 32, -26, -30, -7, -11, -8, 28, 14, 19, -14, 41, 16, 5, -24, -4, -1, -12, 10, -30, 22, 11, 22, -15, 2, -6, 4, 20, -22, 25, 22, 41, 12, 0, 11, -30, -18, -18, -8, -7, 12, 20, -24, -29, -8, 14, 12, 28, 6, 9, 6, -30, -8, 22, -22, 14, 6, -30, -8, -6, -14, 34, -8, 30, 6, -8, -12, -7, -14, -12, 12, 7, -7, 6, -22, 40, 16, -4, -22, -4, -5, 3, -24, 15, -7, -30, -8, 15, 0, -7, 2, -16, -20, 32, -22, 11, -13, 8, -3, -30, 15, -30, -24, -2, 15, 32, -7, 12, 22, -2, -14, -39, 11, 2, -11, 1, -12, -20, 22, 15, 2, 5, 22, 9, 22, 18, 6, -30, 4, 9, 16, 28, -8, -30, -8, 14, -14, 9, -2, -21, 12, -42, 6, 14, 16, 22, 6, -6, 0, 20, 0, -30, 14, -23, -11, 17, -14, 22, 8, -17, 22, -3, -5, -18, 2

In [36]:
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}")
score1 = sorted_results

my_AI vs two_layer_2_AI : -0.63
my_AI vs my_AI : -0.29
my_AI vs two_layer_4_AI : 1.65
my_AI vs two_layer_1_AI : 10.48
my_AI vs two_layer_3_AI : 10.74
my_AI vs one_layer_2_AI : 12.68
my_AI vs one_layer_1_AI : 14.24
my_AI vs three_layer_1_AI : 17.88
my_AI vs random_AI : 26.33
my_AI vs professor_AI : 28.38


In [37]:
playerA = "random_AI"
num_games = 1000  # please use iterations times that is multiples of 12
results = compare_all_ais(playerA, dic, num_games)
# Sorting the match results by average score
sorted_results = sorted(results.items(), key=lambda item: np.mean(item[1]))
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}          [data:{score}]")

1/10 matchups simulated
2/10 matchups simulated
3/10 matchups simulated
4/10 matchups simulated
5/10 matchups simulated
6/10 matchups simulated
7/10 matchups simulated
8/10 matchups simulated
9/10 matchups simulated
10/10 matchups simulated
random_AI vs two_layer_4_AI : -28.05          [data:[-28, -38, -34, -26, -58, -16, -39, -33, -28, -26, -32, 2, 4, -22, -18, -10, -20, -24, -13, -6, -30, 18, -28, -22, -36, -35, -20, -34, -39, -40, -20, -28, -23, -36, -41, -15, -24, -42, -22, -32, -26, -20, -9, -16, -24, -18, -21, -36, -30, -14, -54, -26, -46, -24, -17, -36, -26, -12, -19, -30, -27, -16, -42, -26, -34, -22, -32, -18, -18, -10, -13, -16, -35, -42, -38, -26, -34, -42, -26, -18, -28, -22, -38, -30, -26, -28, -38, -22, -28, -28, -29, -14, -34, -42, -25, -24, -28, -28, -22, -46, -24, -40, -32, -40, -24, -40, -33, -20, -18, -24, -21, -35, -33, -18, -41, -42, -32, -26, -36, -34, -52, -24, -34, -16, -27, -22, -21, -30, -18, -30, -45, -30, -22, -12, -37, -6, -46, -18, -29, 2, -40, -25, -26, -

In [38]:
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}")
score2 = sorted_results

random_AI vs two_layer_4_AI : -28.05
random_AI vs my_AI : -25.90
random_AI vs professor_AI : -17.67
random_AI vs two_layer_3_AI : -14.10
random_AI vs three_layer_1_AI : -13.83
random_AI vs two_layer_1_AI : -12.57
random_AI vs two_layer_2_AI : -11.15
random_AI vs one_layer_2_AI : -10.75
random_AI vs one_layer_1_AI : -10.61
random_AI vs random_AI : -0.39


In [39]:
playerA = "professor_AI"
num_games = 1000  # please use iterations times that is multiples of 12
results = compare_all_ais(playerA, dic, num_games)
# Sorting the match results by average score
sorted_results = sorted(results.items(), key=lambda item: np.mean(item[1]))
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}          [data:{score}]")

1/10 matchups simulated
2/10 matchups simulated
3/10 matchups simulated
4/10 matchups simulated
5/10 matchups simulated
6/10 matchups simulated
7/10 matchups simulated
8/10 matchups simulated
9/10 matchups simulated
10/10 matchups simulated
professor_AI vs two_layer_4_AI : -30.15          [data:[-36, -44, -40, -38, 31, -40, -34, -16, -14, -30, -34, -34, -44, -40, -34, -36, -38, -30, -44, -42, -27, -16, -32, -24, -38, 0, -50, -38, -32, -34, -22, -42, -46, -42, 0, -36, -39, -28, -41, -20, -9, -36, -39, -46, -41, -49, -34, -24, -33, -46, -30, -18, -32, -26, -36, -28, -38, -42, -30, -6, -21, -36, -18, -39, -4, -53, -44, -34, -38, -42, -22, -34, -30, -28, -25, -16, -30, -28, -30, -38, -44, -24, -19, -26, -45, -38, -28, -5, -19, -14, -24, -48, -47, -26, -29, -28, -29, -18, -35, -21, -43, -38, -10, -32, -40, -25, -42, 0, -42, -34, -3, -36, -34, -32, 22, -38, -46, -38, -20, -18, -32, -40, -36, -46, -46, -34, 7, 0, -32, -28, -44, -26, -31, -36, -42, -20, -7, -18, -34, -13, -36, -42, -41, -10, -

In [40]:
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}")
score3 = sorted_results

professor_AI vs two_layer_4_AI : -30.15
professor_AI vs my_AI : -28.14
professor_AI vs two_layer_1_AI : -19.26
professor_AI vs one_layer_2_AI : -17.95
professor_AI vs two_layer_2_AI : -17.86
professor_AI vs one_layer_1_AI : -16.41
professor_AI vs three_layer_1_AI : -12.99
professor_AI vs two_layer_3_AI : -10.89
professor_AI vs professor_AI : -0.21
professor_AI vs random_AI : 16.64


In [41]:
playerA = "one_layer_1_AI"
num_games = 1000  # please use iterations times that is multiples of 12
results = compare_all_ais(playerA, dic, num_games)
# Sorting the match results by average score
sorted_results = sorted(results.items(), key=lambda item: np.mean(item[1]))
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}          [data:{score}]")

1/10 matchups simulated
2/10 matchups simulated
3/10 matchups simulated
4/10 matchups simulated
5/10 matchups simulated
6/10 matchups simulated
7/10 matchups simulated
8/10 matchups simulated
9/10 matchups simulated
10/10 matchups simulated
one_layer_1_AI vs two_layer_1_AI : -31.50          [data:[-30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -30, -33, -

In [42]:
for match, score in sorted_results:
    print(f"{match} : {np.mean(score):.2f}")
score4 = sorted_results

one_layer_1_AI vs two_layer_1_AI : -31.50
one_layer_1_AI vs two_layer_4_AI : -25.00
one_layer_1_AI vs my_AI : -15.08
one_layer_1_AI vs three_layer_1_AI : -2.50
one_layer_1_AI vs one_layer_1_AI : 0.00
one_layer_1_AI vs one_layer_2_AI : 9.00
one_layer_1_AI vs random_AI : 10.63
one_layer_1_AI vs two_layer_3_AI : 12.00
one_layer_1_AI vs two_layer_2_AI : 13.50
one_layer_1_AI vs professor_AI : 15.93


In [43]:
import multiprocessing
import time

def TimeGame(player, random_player):
    board = BoardInit()
    current_player = 1  # Assume 1 is the starting player, for example, Black.
    play_time = 0  # To track the time taken by the player

    while True:
        start_time = time.perf_counter()
        if current_player == 1:
            x, y = player(current_player, board)
        else:
            x, y = random_player(current_player, board)

        if (x, y) == (0, 0):  # No valid move
            if all(PossibleMove((-1) * current_player, board) == []):
                break  # No valid moves for either player, game ends
            current_player *= -1  # Switch player
            continue

        # Place the move on the board
        board = PlaceMove(current_player, board, x, y)
        
        # Calculate time taken for the move
        end_time = time.perf_counter()

        if current_player == 1:
            play_time += end_time - start_time

        # Check if the game should continue
        if PossibleMove((-1) * current_player, board) == []:
            break

        # Switch player
        current_player *= -1

    return play_time

def TimeSimulateGame(args):
    player, random_player = args
    return TimeGame(player, random_player)

def TimeSimulateMultipleGames(player, random_player, num_games):
    total_time = 0

    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        times = pool.map(TimeSimulateGame, [(player, random_player)] * num_games)

    for play_time in times:
        total_time += play_time

    return total_time

In [44]:
num_games = 100
total_times = {}
i = 1

for player_name, player_function in dic.items():
    total_time = TimeSimulateMultipleGames(player_function, random_AI, num_games)
    total_times[player_name] = total_time
    print(f"{i}/{len(dic.items())} matchups simulated")
    i = i + 1
    
avg_times = [(name, time / num_games) for name, time in total_times.items()]
avg_times_sorted = sorted(avg_times, key=lambda x: x[1])  

max_name_length = max(len(name) for name, _ in avg_times_sorted)
print("\nAverage Time Per Game:")
for player_name, avg_time in avg_times_sorted:
    print(f"{player_name.ljust(max_name_length)}: Avg Time: {avg_time:.8f} s")


1/10 matchups simulated
2/10 matchups simulated
3/10 matchups simulated
4/10 matchups simulated
5/10 matchups simulated
6/10 matchups simulated
7/10 matchups simulated
8/10 matchups simulated
9/10 matchups simulated
10/10 matchups simulated

Average Time Per Game:
random_AI       : Avg Time: 0.00218797 s
professor_AI    : Avg Time: 0.00221597 s
one_layer_2_AI  : Avg Time: 0.01759449 s
one_layer_1_AI  : Avg Time: 0.01825536 s
two_layer_1_AI  : Avg Time: 0.03910037 s
two_layer_2_AI  : Avg Time: 0.20943702 s
two_layer_4_AI  : Avg Time: 0.21475568 s
my_AI           : Avg Time: 0.21627552 s
three_layer_1_AI: Avg Time: 0.80990953 s
two_layer_3_AI  : Avg Time: 2.37612846 s
